# Markov-Chain Monte Carlo in EasyVVUQ

We have a model that returns an array of 50 integers where each is supposed to have come from a Poisson distribution with an unknown mean. The model has four input parameters ```a, b, c, d``` and the values of the means depend on the values of those parameters (albeit indirectly).

In [ ]:
import numpy as np
def model(a, b, c, d):
    a = int(a)
    b = int(b)
    c = int(c)
    d = int(d)
    x = np.linspace(0, 1, 50)
    return np.random.poisson(
        a * (0.5 * np.sin(2.0 * np.pi * x) + 1.0) +\
        b * (0.5 * np.sin(4.0 * np.pi * x) + 1.0) +\
        c * (0.5 * np.sin(6.0 * np.pi * x) + 1.0) +\
        d * (0.5 * np.sin(8.0 * np.pi * x) + 1.0))

If we plot the values returned by the model with ```a = b = c = d = 50``` and repeat this 20 times we might get something like the figure below.

In [ ]:
import matplotlib.pyplot as plt
for _ in range(20):
    plt.plot(model(50, 50, 50, 50), '.')

Then we have an array of 50 values that came from the model. With ```a = 50, b = 80, c = 50 and d = 80```. We will treat them as our observed data. We want to know what input parameters of the model are most likely to result in the values in the ```observed``` array. Because suppose we forgot. In most cases this would be values obtained by measuring whatever physical phenomenon the simulation is supposed to model.

In [ ]:
observed = np.array([248, 311, 344, 370, 375, 369, 302, 306, 285, 281, 258, 248, 254,
       273, 292, 268, 312, 269, 230, 244, 227, 219, 247, 233, 225, 238,
       271, 307, 294, 299, 248, 249, 271, 255, 231, 225, 243, 279, 271,
       294, 237, 247, 197, 180, 161, 168, 178, 207, 204, 258])

I'm not a statistician so take the following with a modicum of salt. More precisely we are interested in $\mathbb{E}(X | y)$ with $X = [A, B, C, D]^T$ where $A, B, C, D$ are the random variables representing the input parameters. For this we need to know the probability density $f_{X|y}$ where $y$ are our observed values. From Bayes theorem it follows that $f_{X|y}$ is proportional to $f_{Y|X}(y|X)f(X)$. We usually know $f(X)$ in advance. being the joint probability distribution of the input parameter values given some observed values $x$. We know by Bayes theorem that this is proportional to $E(g(x | a, b, c, d) f(a, b, c, d))$. We will use Markov-Chain Monte Carlo (MCMC) to approximate the probability density $g(x | a, b, c, d)$. Namely the probability density of $x$ taking on given values with respect to the input parameters. We know that $x$ is 50 numbers from 50 Poisson distributions with unknown means. We will estimate those means by running the model multiple times and calculating the sample mean because that is the MLE estimate for a Poisson distributed random variable. Suppose we have $\hat{\lambda}_i$ that is the estimate. Then we have $g(y | a, b, c, d) = \prod_{i = 1}^{50} \frac{\hat\lambda_i^{y_i} \exp(-\hat\lambda_i)}{{y_i}!}$. Since it is often more convenient to work with log-likelihoods we arrive at $\sum_{i = 1}^{50} \left(y_i \log(\hat\lambda_i) - \hat\lambda_i - \log({y_i}!)\right)$. Since this is an estimate special care will need to be taken when estimating convergence of the MCMC algorithm.

In [1]:
import easyvvuq as uq
import scipy.special
import chaospy as cp
import numpy as np
import pandas as pd
import os
import json

In [ ]:
def run_model(directory):
    json_input = os.path.join(directory, 'input.json')
    with open(json_input, "r") as fd:
        inputs = json.load(fd)
    a = inputs['a']
    b = inputs['b']
    c = inputs['c']
    d = inputs['d']
    outfile = inputs['outfile']
    data = pd.DataFrame({"Values": model(a, b, c, d)})
    data.to_csv(os.path.join(directory, outfile), index=False)

In [ ]:
def poisson_loglikelihood(lmbda, k):
    return (k * np.log(lmbda) - lmbda - scipy.special.gammaln(k + 1.0)).sum()

In [ ]:
def get_likelihood(observed):
    def likelihood(lmbda):
        return poisson_loglikelihood(lmbda, observed)
    return likelihood

In [ ]:
params = {
    "a": {"type": "float", "min": 0, "max": 100, "default": 50},
    "b": {"type": "float", "min": 0, "max": 100, "default": 50},
    "c": {"type": "float", "min": 0, "max": 100, "default": 50},
    "d": {"type": "float", "min": 0, "max": 100, "default": 50},
    "outfile": {"type": "string", "default": "output.csv"},
    "replica_id": {"type": "integer", "default": 0},
    "chain_id": {"type": "integer", "default": 0}
}
encoder = uq.encoders.GenericEncoder(template_fname="mcmc.template", 
                                     delimiter="$", target_filename="input.json")
decoder = uq.decoders.SimpleCSV("output.csv", ["Values"])
campaign = uq.Campaign(
    name="mcmc", encoder=encoder, decoder=decoder, 
    params=params, work_dir='.')

In [ ]:
vary_init = {
    "a": np.random.uniform(0, 100, size=5),
    "b": np.random.uniform(0, 100, size=5),
    "c": np.random.uniform(0, 100, size=5),
    "d": np.random.uniform(0, 100, size=5)
}

In [ ]:
def q(x, b=10):
    return cp.J(cp.Normal(x['a'], b), 
                cp.Normal(x['b'], b), 
                cp.Normal(x['c'], b), 
                cp.Normal(x['d'], b))

In [ ]:
mcmc_sampler = uq.sampling.MCMCSampler(
    vary_init, q, 'Values', n_chains=5, n_replicas=20,
    likelihood=get_likelihood(observed),
    replica_col="replica_id", estimator=lambda x: np.mean(x), 
    minimum_probability=0.0)
sampler = uq.sampling.ReplicaSampler(mcmc_sampler)
campaign.set_sampler(sampler)

In [ ]:
for _ in range(1000):
    campaign.sample_and_apply(uq.actions.ExecutePython(run_model), 
                              nsamples=100, mark_invalid=True).start()
    campaign.collate()
    sampler.update(campaign)

Let us now save our progress so that we do not lose it. This way we can continue the tutorial just from this point onward and not redo the sampling stage.

In [ ]:
campaign.save_state('mcmc_state.json')

In [2]:
campaign = uq.Campaign(state_file='mcmc_state.json')

In [4]:
df = campaign.get_collation_result()

In [18]:
df[df[('chain_id', 0)] == 0]

run_id collation          a          b          c          d chain_id  \
         0         0          0          0          0          0        0    
0         1        -1  24.911009  75.929047  26.736209  41.919606        0   
5         6        -1  24.911009  75.929047  26.736209  41.919606        0   
10       11        -1  24.911009  75.929047  26.736209  41.919606        0   
15       16        -1  24.911009  75.929047  26.736209  41.919606        0   
20       21        -1  24.911009  75.929047  26.736209  41.919606        0   
...     ...       ...        ...        ...        ...        ...      ...   
4255  98476        -1  45.325790  80.708075  56.147346  82.762336        0   
4256  98481        -1  45.325790  80.708075  56.147346  82.762336        0   
4257  98486        -1  45.325790  80.708075  56.147346  82.762336        0   
4258  98491        -1  45.325790  80.708075  56.147346  82.762336        0   
4259  98496        -1  45.325790  80.708075  56.147346  82.762336        0   

     replica_id     outfile Values  ...                                     \
             0           0      0   ...     40     41     42     43     44   
0             0  output.csv  179.0  ...  159.0  135.0  135.0  125.0   99.0   
5             0  output.csv  150.0  ...  168.0  131.0  132.0  122.0  100.0   
10            0  output.csv  147.0  ...  127.0  135.0  124.0  101.0  103.0   
15            0  output.csv  178.0  ...  141.0  159.0  124.0  109.0  115.0   
20            0  output.csv  150.0  ...  150.0  136.0  122.0  112.0  106.0   
...         ...         ...    ...  ...    ...    ...    ...    ...    ...   
4255          0  output.csv  265.0  ...  237.0  235.0  188.0  184.0  175.0   
4256          0  output.csv  252.0  ...  255.0  212.0  203.0  177.0  194.0   
4257          0  output.csv  243.0  ...  232.0  232.0  212.0  192.0  172.0   
4258          0  output.csv  253.0  ...  256.0  226.0  232.0  186.0  157.0   
4259          0  output.csv  256.0  ...  238.0  251.0  207.0  185.0  163.0   

                                         
         45     46     47     48     49  
0      81.0   94.0  142.0  136.0  146.0  
5     111.0  120.0  109.0  134.0  182.0  
10    119.0  108.0  104.0  135.0  152.0  
15    100.0  118.0  124.0  135.0  156.0  
20     93.0  102.0  114.0  148.0  174.0  
...     ...    ...    ...    ...    ...  
4255  138.0  134.0  192.0  229.0  244.0  
4256  156.0  183.0  186.0  230.0  272.0  
4257  169.0  157.0  199.0  200.0  287.0  
4258  172.0  162.0  197.0  231.0  259.0  
4259  151.0  159.0  212.0  230.0  296.0  

[1120 rows x 59 columns]